In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
from sklearn.decomposition import PCA
import pickle
pd.set_option("display.max_columns", None)

In [9]:
store = pd.HDFStore('predict_data.h5')
df = store['predict_data']


In [10]:
filename = 'lgbm_model_default.sav'
default_model = pickle.load(open(filename, 'rb'))

In [11]:
filename = 'lgbm_model_prepay.sav'
prepay_model = pickle.load(open(filename, 'rb'))

In [12]:
df

,unique_id,Orig_year,Orig_month,loan_amount,monthly_payment,grade,income,inq_last_6m,num_open_accts,revol_bal,num_tot_accts,tot_credit_bal,accts_opn_last_6m,inq_last_12m,accts_opn_last_24m,employment_length,home_ownership,int_rate,earliest_cr_line,fico,util_rate,loan_over_income,term,dti,platform_id,post_dti
0,A-223264-1,2016,6,28000.0,779.56,E,108000.0000,0.745387,10.0,14040.0,19.0,236454.0,29.927927,2.0,5.0,8,MORTGAGE/OWN,22.39,194.0,682.0,63.2,0.260000,60,0.110900,A,0.197517
1,A-753340-2,2017,6,22000.0,496.30,C,80000.0000,1.000000,18.0,21846.0,42.0,122694.0,3.000000,2.0,11.0,0,MORTGAGE/OWN,12.62,209.0,702.0,46.0,0.280000,60,0.171300,A,0.245744
2,A-640827-3,2016,4,7000.0,210.81,A,45000.0000,0.745387,9.0,5307.0,35.0,72196.0,1.000000,5.0,6.0,10,MORTGAGE/OWN,5.32,96.0,687.0,39.0,0.160000,36,0.053300,A,0.109515
3,A-861547-4,2016,11,15000.0,487.47,B,83000.0000,0.745387,11.0,11443.0,29.0,56920.0,29.927927,2.0,4.0,10,RENT,10.49,87.0,672.0,92.3,0.180000,36,0.312600,A,0.383077
4,A-244739-5,2017,7,14400.0,467.50,B,55000.0000,0.745387,7.0,4425.0,12.0,202510.0,29.927927,5.0,4.0,5,MORTGAGE/OWN,10.42,203.0,667.0,15.5,0.260000,36,0.212500,A,0.314498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139546,C-932830-40302,2018,12,21000.0,648.80,AA,72000.0000,1.000000,7.0,226.0,14.0,212426.0,29.927927,1.0,2.0,1,MORTGAGE/OWN,7.04,22.0,830.0,8.0,0.291667,36,0.071868,C,0.180000
139547,C-758002-40303,2018,12,3500.0,134.83,D,52800.0000,1.000000,22.0,2363.0,46.0,83792.0,3.000000,3.0,15.0,4,RENT,22.64,125.0,690.0,27.0,0.066288,36,0.409357,C,0.440000
139548,C-776941-40304,2018,12,10000.0,349.80,C,54999.9996,1.000000,12.0,4651.0,21.0,9482.0,3.000000,3.0,6.0,0,RENT,15.64,166.0,670.0,16.0,0.181818,36,0.183681,C,0.260000
139549,C-694817-40305,2018,12,12000.0,335.81,D,45585.9996,7.000000,19.0,733.0,31.0,72949.0,5.000000,7.0,1.0,6,MORTGAGE/OWN,22.64,376.0,770.0,1.0,0.263239,60,0.301604,C,0.390000


In [13]:
appended_data = []
for chunk in np.array_split(df, 2):
    df_rep = chunk.iloc[np.repeat(np.arange(len(chunk)), chunk["term"])]
    
    df_mob = pd.DataFrame(df_rep, columns= ['unique_id'])
    df_mob['mob'] = df_mob.groupby("unique_id").cumcount()+1
    
    df_combine = pd.concat([df_rep, df_mob], axis=1)
    X = df_combine.drop(['unique_id'], axis=1)
    
    encoder_1 = LabelEncoder()
    # Fit the encoder on the data "platform_id"
    encoder_1.fit(X["platform_id"])
    X_platform_id_encoded = encoder_1.transform(X["platform_id"])
    X["platform_id"] = X_platform_id_encoded

    # Fit the encoder on the data "grade"
    encoder_2 = LabelEncoder()
    encoder_2.fit(X["grade"])
    X_grade_encoded = encoder_2.transform(X["grade"])
    X["grade"] = X_grade_encoded

    # Fit the encoder on the data "home_ownership"
    encoder_3 = LabelEncoder()
    encoder_3.fit(X["home_ownership"])
    X_home_ownership_encoded = encoder_3.transform(X["home_ownership"])
    X["home_ownership"] = X_home_ownership_encoded
    
    X_mob = X['mob'].reset_index(drop=True)
    X = X.drop('mob', axis=1)
    X_year = X['Orig_year'].reset_index(drop=True)
    X = X.drop('Orig_year', axis=1)
    X_month = X['Orig_month'].reset_index(drop=True)
    X = X.drop('Orig_month', axis=1)    
    
    
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X)

    # Apply PCA
    pca = PCA(n_components= 15)
    X_pca = pd.DataFrame(pca.fit_transform(X_scaled))
    
    X_pca = pd.concat([X_pca, X_mob,X_year,X_month], axis=1)
    # predict probabilities
    default_probs = default_model.predict_proba(X_pca)
    # keep probabilities for the positive outcome only
    default_probs = default_probs[:, 1]
    df_combine['CO_Probability'] = default_probs
    
    prepay_probs = prepay_model.predict_proba(X_pca)
    # keep probabilities for the positive outcome only
    prepay_probs = prepay_probs[:, 1]
    df_combine['PrePmt_Probability'] = prepay_probs
    
    appended_data.append(df_combine[['unique_id','mob','CO_Probability','PrePmt_Probability']].reset_index(drop=True))
    print("ok")
appended_data = pd.concat(appended_data)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

ok


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

ok


In [14]:
prediction = appended_data.loc[:,~appended_data.columns.duplicated()]

In [16]:
prediction[prediction['unique_id']=='A-244739-5']

,unique_id,mob,CO_Probability,PrePmt_Probability
192,A-244739-5,1,0.000001,0.009089
193,A-244739-5,2,0.000001,0.009246
194,A-244739-5,3,0.000027,0.009405
195,A-244739-5,4,0.000195,0.009667
196,A-244739-5,5,0.001627,0.010215
197,A-244739-5,6,0.003045,0.010962
198,A-244739-5,7,0.004341,0.012489
199,A-244739-5,8,0.005222,0.012621
200,A-244739-5,9,0.005918,0.013260
201,A-244739-5,10,0.006152,0.014422


In [22]:
prediction.to_csv('prediction.csv', index = False)